# Atomic and molecular integral with pySCF

In [ ]:
import numpy as np
from pyscf import gto, scf, ao2mo

In [ ]:
import qib

### Construct the molecule

In [ ]:
atom = '''H 0 0 0; H 0 0 0.735'''
basis = 'sto-6g'
units = 'angstrom'
charge = 0
spin = 0
verbose = 0

mol = gto.Mole()
mol.build(atom    = atom,
          basis   = basis,
          charge  = charge,
          spin    = spin,
          units   = units,
          verbose = verbose)

### Create the atomic integrals

In [ ]:
# costant energy term (nuclear repulsion and kinetic term)
h0 = mol.get_enuc()
# one body integral in the spin-orbital basis (electronic kinetic term and electronic-nuclear attraction)
h1 = np.kron(mol.get_hcore(), np.identity(2))
# two body integral in the spin-orbital basis (electronic repulsion)
h2 = mol.intor('int2e_spinor')

# create a MolecularHamiltonian object starting from the atomic integrals
latt = qib.lattice.LayeredLattice(qib.lattice.FullyConnectedLattice((2,)), 2)
field = qib.field.Field(qib.field.ParticleType.FERMION, latt)
H = qib.operator.MolecularHamiltonian(field, h0, h1, h2.transpose(0,2,3,1))

In [ ]:
print(H.as_matrix())

### Hartree-Fock calculation

In [ ]:
# RHF == restricted HF (look also ROHF, UHF...)
mf = scf.RHF(mol)
# performs the HF calculation
mf.kernel()
# gets the coefficient matrix for the molecular orbitals
coeff = mf.mo_coeff
print(coeff)

### Molecular integrals

In [ ]:
spin_coeff = np.kron(coeff, np.identity(2))
h1_mo = np.einsum('ji,jk,kl->il', spin_coeff.conj(), h1, spin_coeff)
h2_mo = np.einsum('pqrs,pi,qj,rk,sl->ijkl', h2, spin_coeff, spin_coeff, spin_coeff, spin_coeff)
print(h1_mo)
print(h2_mo)

### Jordan-Wigner mapping

In [ ]:
pauli_op = jordan_wigner_encode_field_operator(H.as_field_operator())

# We must assign a lattice to the pauli operator and the number of sites in the lattice must be exactly like the size of the Pauli strings
field_q = Field(qib.field.ParticleType.QUBIT, qib.lattice.LayeredLattice(qib.lattice.FullyConnectedLattice((2,)), 2))
pauli_op.set_field(field_q)
print(pauli_op)